In [12]:
import sys

from pathlib import Path
from datetime import timedelta

import dateutil
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf
import keras

Using TensorFlow backend.


In [2]:
df = pd.read_csv("C://workspace//project_data//lata//covid.csv")
df.head()

,id,sex,patient_type,entry_date,date_symptoms,date_died,intubed,pneumonia,age,pregnancy,...,inmsupr,hypertension,other_disease,cardiovascular,obesity,renal_chronic,tobacco,contact_other_covid,covid_res,icu
0,16169f,2,1,04-05-2020,02-05-2020,9999-99-99,97,2,27,97,...,2,2,2,2,2,2,2,2,1,97
1,1009bf,2,1,19-03-2020,17-03-2020,9999-99-99,97,2,24,97,...,2,2,2,2,2,2,2,99,1,97
2,167386,1,2,06-04-2020,01-04-2020,9999-99-99,2,2,54,2,...,2,2,2,2,1,2,2,99,1,2
3,0b5948,2,2,17-04-2020,10-04-2020,9999-99-99,2,1,30,97,...,2,2,2,2,2,2,2,99,1,2
4,0d01b5,1,2,13-04-2020,13-04-2020,22-04-2020,2,2,60,2,...,2,1,2,1,2,2,2,99,1,2


In [3]:
df.describe()

,sex,patient_type,intubed,pneumonia,age,pregnancy,diabetes,copd,asthma,inmsupr,hypertension,other_disease,cardiovascular,obesity,renal_chronic,tobacco,contact_other_covid,covid_res,icu
count,566602.000000,566602.000000,566602.000000,566602.000000,566602.000000,566602.000000,566602.000000,566602.000000,566602.000000,566602.000000,566602.000000,566602.000000,566602.000000,566602.000000,566602.000000,566602.000000,566602.000000,566602.000000,566602.000000
mean,1.506726,1.215165,76.562952,1.846262,42.622483,50.400692,2.210633,2.280221,2.265029,2.319628,2.145774,2.410030,2.286157,2.138905,2.283765,2.238360,31.573034,1.728651,76.562864
std,0.499955,0.410937,39.058676,0.560939,16.659973,47.501579,5.683523,5.327832,5.334658,5.667381,5.459866,6.489959,5.438405,5.395578,5.393232,5.571901,45.082123,0.658710,39.059060
min,1.000000,1.000000,1.000000,1.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
25%,1.000000,1.000000,97.000000,2.000000,31.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,1.000000,1.000000,97.000000
50%,2.000000,1.000000,97.000000,2.000000,41.000000,97.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,97.000000
75%,2.000000,1.000000,97.000000,2.000000,53.000000,97.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,99.000000,2.000000,97.000000
max,2.000000,2.000000,99.000000,99.000000,120.000000,98.000000,98.000000,98.000000,98.000000,98.000000,98.000000,98.000000,98.000000,98.000000,98.000000,98.000000,99.000000,3.000000,99.000000


## 외래 환자 제거
## 남성 임신 97을 임신 아님 2로 변경

In [4]:
train_df = df.drop(df.loc[df['patient_type'] == 1].index)

train_df['pregnancy'].replace(97, 2, inplace=True)
train_df['pregnancy'].value_counts()

2     120676
1       1071
98       166
Name: pregnancy, dtype: int64

## 97,98,99 결측치 제거

In [5]:
train_df.iloc[:,6:] = train_df.iloc[:,6:].replace([97,98,99],np.nan)
train_df.dropna(inplace=True)
#train_df.iloc[:,6:].value_counts()

"""
#check = []
#for i in train_df.iloc[:,6:]:
#    check.append(train_df[train_df[i]==98])

#print(check)
"""

'\n#check = []\n#for i in train_df.iloc[:,6:]:\n#    check.append(train_df[train_df[i]==98])\n\n#print(check)\n'

## 중복된 id 제거

In [6]:
#temp = []
#train_df['id'] = train_df['id'].astype(str)

# 중복된 id들은 모두 다른사람들이긴 하다. 이것을 지워야 하는지 논의해보자

## 생존/사망 (0, 1) 분류

In [7]:
train_df['dd_bool'] = np.where(train_df['date_died'] == '9999-99-99', 0, 1)
train_df['dd_bool'].value_counts()

0    46578
1    12022
Name: dd_bool, dtype: int64

## 증상 발현 후 입원에서부터 사망까지 시간

우선은 사망률을 예측 분석하는 것이니까 살아있는 사람들은 고려를 하지 않아 보았다

In [8]:
train_df.entry_date = pd.to_datetime(df.entry_date, dayfirst=True, errors='coerce')
train_df.date_symptoms = pd.to_datetime(df.date_symptoms, dayfirst=True, errors='coerce')
train_df.date_died = pd.to_datetime(df.date_died, dayfirst=True, errors='coerce')
train_df['dura_of_d'] = train_df['date_died'] - train_df['entry_date']

In [9]:
train_df

,id,sex,patient_type,entry_date,date_symptoms,date_died,intubed,pneumonia,age,pregnancy,...,other_disease,cardiovascular,obesity,renal_chronic,tobacco,contact_other_covid,covid_res,icu,dd_bool,dura_of_d
21,154fea,1,2,2020-06-02,2020-06-02,NaT,2.0,2.0,25.0,2.0,...,2.0,2.0,2.0,2.0,2.0,1.0,1,2.0,0,NaT
30,16e14e,1,2,2020-06-22,2020-06-17,NaT,2.0,2.0,52.0,2.0,...,2.0,2.0,1.0,2.0,1.0,1.0,1,2.0,0,NaT
50,03f45a,2,2,2020-04-12,2020-04-07,2020-04-13,1.0,1.0,57.0,2.0,...,2.0,2.0,1.0,2.0,1.0,2.0,1,2.0,1,1 days
71,149f1c,1,2,2020-06-17,2020-06-12,NaT,2.0,1.0,51.0,2.0,...,2.0,2.0,2.0,2.0,2.0,1.0,1,2.0,0,NaT
79,1d5df2,1,2,2020-06-08,2020-06-07,NaT,1.0,1.0,67.0,2.0,...,2.0,2.0,1.0,2.0,2.0,1.0,1,2.0,0,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
566559,1d5382,1,2,2020-06-21,2020-06-16,NaT,2.0,2.0,17.0,2.0,...,2.0,2.0,2.0,2.0,2.0,2.0,3,2.0,0,NaT
566565,001dc8,2,2,2020-06-25,2020-06-25,NaT,2.0,1.0,68.0,2.0,...,2.0,2.0,1.0,2.0,2.0,2.0,3,2.0,0,NaT
566578,028be6,1,2,2020-06-05,2020-05-23,2020-06-14,1.0,1.0,34.0,2.0,...,2.0,2.0,2.0,2.0,2.0,1.0,3,2.0,1,9 days
566590,067bbd,2,2,2020-06-07,2020-05-24,NaT,1.0,1.0,68.0,2.0,...,2.0,2.0,1.0,2.0,2.0,2.0,3,1.0,0,NaT


In [10]:
# ''' '''사이의 코드는 텐서후로우 1 버전 환경으로 작성한것입니다
# 배열이 맞지않아 작동하지 않습니다

'''

tf.set_random_seed(777)

x_train = train_df[['sex', 'intubed','pneumonia','age','pregnancy', 'diabetes', 'copd', 'asthma','inmsupr','hypertension','other_disease','cardiovascular','obesity', 'renal_chronic', 'tobacco', 'contact_other_covid', 'covid_res', 'icu']]
y_train = train_df['dd_bool']

#x = tf.placeholder(tf.float32, shape=[58600, None])
#y = tf.placeholder(tf.float32, shape=[1, None])

#w = tf.Variable(tf.random_normal([1, 58600]), name='weight')
#b = tf.Variable(tf.random_normal([1]), name='bias')

x = tf.constant(dataVar, dtype=tf.float32, shape=[58600, 25])
y = tf.constant()


hypothesis = tf.sigmoid(tf.matmul(x,w) + b)

loss = - tf.reduce_mean(y * tf.log(hypothesis) + (1-y) * tf.log(1-hypothesis))

optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.01)
train = optimizer.minimize(loss)

predict = tf.cast(hypothesis > 0.5, dtype=tf.float32)
accuracy = tf.reduce_mean(tf.cast(tf.equal(predict, y), dtype=tf.float32))

sess = tf.Session()

'''

## 텐서후로우 2 버전으로 작성된 코드 입니다

x_train = train_df[['sex', 'intubed','pneumonia','age','pregnancy', 'diabetes', 'copd', 'asthma','inmsupr','hypertension','other_disease','cardiovascular','obesity', 'renal_chronic', 'tobacco', 'contact_other_covid', 'covid_res', 'icu']]
target = train_df['dd_bool']

#y_train = train_df['dd_bool']
#target = train_df.pop('dd_bool')

dataset = tf.data.Dataset.from_tensor_slices((x_train.values, target.values))

for feat, targ in dataset.take(5):
    print('Festures: {}, Target: {}'.format(feat, targ))


Festures: [ 1.  2.  2. 25.  2.  2.  2.  2.  2.  2.  2.  2.  2.  2.  2.  1.  1.  2.], Target: 0
Festures: [ 1.  2.  2. 52.  2.  2.  2.  2.  2.  2.  2.  2.  1.  2.  1.  1.  1.  2.], Target: 0
Festures: [ 2.  1.  1. 57.  2.  2.  2.  2.  2.  2.  2.  2.  1.  2.  1.  2.  1.  2.], Target: 1
Festures: [ 1.  2.  1. 51.  2.  2.  2.  2.  2.  2.  2.  2.  2.  2.  2.  1.  1.  2.], Target: 0
Festures: [ 1.  1.  1. 67.  2.  1.  2.  2.  2.  1.  2.  2.  1.  2.  2.  1.  1.  2.], Target: 0


In [38]:
"""
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())

    for step in range(10001):
        _, loss_val = sess.run([train, loss], feed_dict={x:x_train, y:y_train})

        if step % 200 == 0:
            print(step, loss_val)

    h,p,a = sess.run([hypothesis, predict, accuracy], feed_dict={x:x_train, y:y_train})

    print("\n예측값:\n", h, "\npredict:\n", p, "\n정확도:",a)
    
"""

train_dataset = dataset.shuffle(len(train_df)).batch(1)

def get_compiled_model():
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(58600, activation='sigmoid'),
        tf.keras.layers.Dense(5, activation='sigmoid'),
        tf.keras.layers.Dense(1)
    ])
    
    model.compile(optimizer='adam',
                 loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
                 metrics=['accuracy'])
    return model

In [39]:
model = get_compiled_model()
model.fit(train_dataset, epochs=1)

Train for 58600 steps
58600/58600 [==============================] - 323s 6ms/step - loss: 0.5080 - accuracy: 0.7948
